In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# change path
import os

try:
  project_fname = '/content/drive/My Drive/Colab Notebooks/c247/'
  os.chdir(project_fname)
except:
  project_fname='/content/drive/My Drive/c247'
  os.chdir(project_fname)
project_data_path = os.path.join(project_fname,'project_data/')
print(os.getcwd())

/content/drive/My Drive/Colab Notebooks/c247


In [ ]:
# load data
import numpy as np
import random
from load_data import * 
from data_preprocessing import * 
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn import preprocessing

X_test, y_test, person_train_valid, X_train_valid, y_train_valid, person_test = load_data(dir_path = project_data_path)

# normalize the data
N_trials,N_eeg,N_bins,_ = X_train_valid.shape
X_train_valid_norm = np.reshape(preprocessing.scale(np.reshape(X_train_valid,(N_trials*N_eeg,N_bins)),axis=1),(N_trials,N_eeg,N_bins,1))
N_trials,N_eeg,N_bins,_ = X_test.shape
X_test_norm = np.reshape(preprocessing.scale(np.reshape(X_test,(N_trials*N_eeg,N_bins)),axis=1),(N_trials,N_eeg,N_bins,1))


print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

In [ ]:
from sklearn import preprocessing
from scipy import signal 
import operator

# Data preprocessing

# Subsample and split into 1 person and 5 EEGs
subsample = 5
subj = [5]
eegs = [0,7,9,11,19]

X_train_valid_subsample, y_train_valid_subsample, person_train_valid_subsample = subsample_data(X_train_valid_norm,y_train_valid, person_train_valid, sample_every=subsample)
X_train, y_train, person_train = split_data_by_subject(X_train_valid_subsample, y_train_valid_subsample, person_train_valid_subsample)

#only get wanted subjects from X_train rather than have all subjects
X_train = operator.itemgetter(*subj)(X_train)
y_train = operator.itemgetter(*subj)(y_train)

X_train = X_train[:,eegs,:,:]

print('Shapes: x = {}, y = {}'.format(X_train.shape, y_train.shape))

# wavelet transform 
N_trials,N_eeg,N_bins,_ = X_train.shape
fs = 250
freq_bins = 50
X_train_cwt = morlet_wavelet_transform(X_train,fs=fs,freq_range=(1,20),freq_bins=freq_bins,w=6)
# reshape to input to CNN
X_train_cwt = np.swapaxes(np.swapaxes(X_train_cwt,1,3),1,2)
# scale between -1 and 1 too mimic output of generator
X_train_cwt_norm = 2 * (X_train_cwt - np.min(X_train_cwt,axis=0) ) / (np.max(X_train_cwt,axis=0) - np.min(X_train_cwt,axis=0)) - 1

print('X_train_cwt_norm = {}'.format(X_train_cwt_norm.shape))

print('Deleting original training/validation data from memory...')
del X_train_valid_subsample, y_train_valid_subsample, person_train_valid_subsample,\
    X_train, person_train, X_train_valid, y_train_valid, person_train_valid

In [ ]:
#load the artificial data and append all of it to the training data
split_art_data = 4
for task in range(4):
  pickle_file = os.path.join(project_data_path,'X_artificial_task{}_subj5_eegs5.npy'.format(task))
  trials = np.load(pickle_file)
  N=trials.shape[0]
  X_train_cwt_norm = np.append(X_train_cwt_norm,trials[:N//split_art_data],axis=0)
  y_train = np.append(y_train,np.ones((N//split_art_data))*task)
print(X_train_cwt_norm.shape)
print(y_train.shape)

In [ ]:
#Convert test data with CWT exactly the same as X_train_valid

X_test_subsample, y_test_subsample, person_test_subsample = subsample_data(X_test_norm,y_test, person_test, sample_every=subsample)
X_test, y_test, person_test = split_data_by_subject(X_test_subsample, y_test_subsample, person_test_subsample)

#get the specific subjects we want by indexing with the operator module
X_test = operator.itemgetter(*subj)(X_test)
y_test = operator.itemgetter(*subj)(y_test)
X_test = X_test[:,eegs,:,:]

# wavelet transform 
N_trials,N_eeg,N_bins,_ = X_test.shape
fs = 250
freq_bins = 50
X_test_cwt = morlet_wavelet_transform(X_test,fs=fs,freq_range=(1,20),freq_bins=freq_bins,w=6)
# reshape to put in same format as the WGAN output
X_test_cwt = np.swapaxes(np.swapaxes(X_test_cwt,1,3),1,2)
# scale between -1 and 1
X_test_cwt_norm = 2 * (X_test_cwt - np.min(X_test_cwt,axis=0) ) / (np.max(X_test_cwt,axis=0) - np.min(X_test_cwt,axis=0)) - 1

print('X_cwt_norm = {}'.format(X_test_cwt_norm.shape))

In [ ]:

%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
#import all needed modules
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Input,Dense,Conv2D,ReLU,ELU,\
  Activation,Flatten,AveragePooling2D,Softmax,BatchNormalization,MaxPooling2D,\
  Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,\
  ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from datetime import datetime


CNN for CWT Dataset

In [ ]:
#(50,200,22)
def create_model():
  model = Sequential()
  model.add(Conv2D(filters=32,
                   kernel_size=(7,7),
                   data_format='channels_last',
                   kernel_regularizer=regularizers.l2(0.01),
                   activation='relu',
                   kernel_initializer='lecun_uniform',
                   input_shape=(50,200,5))) #(42,192,32)
  model.add(BatchNormalization(axis=-1))
  model.add(MaxPooling2D())#(21,96,32)
  model.add(Dropout(rate=0.5))
  model.add(Conv2D(filters=32,
                   kernel_size=(7,7),
                   data_format='channels_last',
                   kernel_regularizer=regularizers.l2(0.01),
                   activation='relu',
                   kernel_initializer='lecun_uniform'))
  model.add(BatchNormalization(axis=-1))
  model.add(MaxPooling2D()) 
  model.add(Dropout(rate=0.5))
  model.add(Flatten())
  model.add(Dense(750,
                  kernel_regularizer=regularizers.l2(0.01),
                  activation='relu',
                  kernel_initializer='lecun_uniform'))
  model.add(Dense(num_classes,
                  activation='softmax'))
  return model

rand_seed = int(datetime.strftime(datetime.now(),"%Y%m%d%H%M%S"))
batch_size=32
num_folds=5
num_classes=4

In [ ]:
optimizer = Adam(learning_rate=1e-5)
try:
  del model
  K.clear_session()
except Exception as e:
  print(e)

model = create_model()
model.compile(loss=categorical_crossentropy,optimizer=optimizer,metrics=['accuracy'])
#plot_model(model,'cnn_cwt_architecture.png',show_shapes=True)

In [ ]:

#lr_plateau = ReduceLROnPlateau(patience=4)
history = model.fit(x=X_train_cwt_norm,
          y=to_categorical(y_train),
          epochs=30,
          #validation_split=1/num_folds,
          validation_data=(X_test_cwt_norm,to_categorical(y_test)),
          batch_size=batch_size,
          callbacks=[])

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()